In [ ]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models,Sequential
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

import tensorflow_addons as tfa

import cv2, numpy as np

### It rewrite the notebook "https://www.kaggle.com/vijayshankar756/inception-rethink" from using Siquential input keras to Functional API. <br>
### You can use multiple Deep Neural Network model this notebook. <br>
### please upvote "https://www.kaggle.com/vijayshankar756/inception-rethink" notebook. <br>
### It simple and easy to understand code! <br>

In [ ]:
train_data_csv = '../input/petfinder-pawpularity-score/train.csv'
test_data_csv  = '../input/petfinder-pawpularity-score/test.csv'
train_folder = '../input/petfinder-pawpularity-score/train'
test_folder  = '../input/petfinder-pawpularity-score/test'

## Hyperparameter for our newtowk

In [ ]:
## parameter setting
y = 'Pawpularity'
# validation_size = 0.3
autotune = tf.data.experimental.AUTOTUNE
img_size = 299
batch_size = 8
dropout = 0.3
lr = 1e-3
decay_steps = 100
decay_rate = 0.95
epochs = 100
KFOLD = 3

In [ ]:
def id_to_path(img_id,dir):
    return os.path.join(dir, f'{img_id}.jpg')

In [ ]:
def get_image(path):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, img_size, img_size), dtype=tf.float32)
    return tf.keras.applications.inception_resnet_v2.preprocess_input(image)

In [ ]:
def process_dataset(path, label):
    return get_image(path), label

In [ ]:
def get_dataset(x, y=None):
    if y is not None:
        ds = tf.data.Dataset.from_tensor_slices((x, y))
        return ds.map(process_dataset, num_parallel_calls=autotune) \
            .batch(batch_size).prefetch(buffer_size=autotune)
    else:
        ds = tf.data.Dataset.from_tensor_slices(x)
        return ds.map(get_image, num_parallel_calls=autotune) \
            .batch(batch_size).prefetch(buffer_size=autotune)

In [ ]:
data_train = pd.read_csv(train_data_csv)
data_train.head()

In [ ]:
data_test = pd.read_csv(test_data_csv)
data_test.head()

In [ ]:
data_train['path'] = data_train['Id'].apply(lambda x: id_to_path(x, train_folder))
data_test['path'] = data_test['Id'].apply(lambda x: id_to_path(x, test_folder))

# train_subset, valid_subset = train_test_split(
#     data_train[['path', y]],
#     test_size=validation_size, shuffle=True, random_state=777
# )

skf = KFold(n_splits=KFOLD, random_state=777, shuffle=True)
data_train['fold'] = -1
for nth, (_, valid_index) in enumerate(skf.split(data_train)):
    data_train.loc[valid_index, 'fold'] = nth

In [ ]:
data_train.head()

In [ ]:
data_train.path

In [ ]:
data_train

In [ ]:
data_train.fold.value_counts()

In [ ]:
model = tf.keras.models.load_model('../input/keras-applications-models/InceptionResNetV2.h5')
model.trainable = False

In [ ]:
inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3))
x = tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical")(inputs)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)(x)
x = tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)(x)
x = tf.keras.layers.experimental.preprocessing.RandomHeight(0.2)(x)
x = tf.keras.layers.experimental.preprocessing.RandomWidth(0.2)(x)
# x = tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)(x)
x = model(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(dropout, name='top_dropout')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(dropout, name='second_dropout')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=decay_steps, decay_rate=decay_rate,staircase=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true*100 - y_pred*100)))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                    loss=rmse,
                    metrics=[rmse])

In [ ]:
##modeling
prediction_lists = pd.DataFrame()
for i in range(0,KFOLD):
    train_subset = data_train[data_train['fold'] != i]
    train_subset = train_subset[['path',y]]
    valid_subset = data_train[data_train['fold'] == i]
    valid_subset = valid_subset[['path',y]]
    train_data = get_dataset(x=train_subset['path'], y=train_subset[y]/100.)
    valid_data = get_dataset(x=valid_subset['path'], y=valid_subset[y]/100.)
    test_data = get_dataset(x=data_test['path'])

    history=model.fit(train_data,validation_data=valid_data, epochs=epochs,verbose=1,callbacks=[early_stop],use_multiprocessing=True, workers=-1)
    history_frame = pd.DataFrame(history.history)
    history_frame.loc[:, ['loss', 'val_loss']].plot()
    history_frame.loc[:, ['rmse', 'val_rmse']].plot()

    ##prediction
    data_test[y+str(i)] = model.predict(test_data,use_multiprocessing=True, workers=os.cpu_count())*100
    data_test[y+str(i)] = data_test[y+str(i)].round()
    prediction_lists = pd.concat([prediction_lists,data_test[[y+str(i)]]],axis=1)

In [ ]:
# import pandas as pd

# history_frame = pd.DataFrame(history.history)

# history_frame.loc[:, ['loss', 'val_loss']].plot()
# history_frame.loc[:, ['rmse', 'val_rmse']].plot()

In [ ]:
prediction_lists

In [ ]:
prediction_lists.mean(axis=1)

In [ ]:
data_test[y] = prediction_lists.mean(axis=1).round()
data_test[['Id', y]].to_csv('submission.csv', index=False)
data_test[['Id', y]].head()

In [ ]:
# ##prediction
# data_test[y] = model.predict(test_data,use_multiprocessing=True, workers=os.cpu_count())*100
# data_test[y] = data_test[y].round()

In [ ]:
###
def InputData():
    pass

def Modeling():
    pass

def Prediction():
    pass

def main():
    InputData()
    
    Modeling()
    
    Prediction()
